# Revenue Management Simple Two-Class Model
## Littlewood's Rule

Reference: https://link.springer.com/article/10.1057/palgrave.rpm.5170134

The Littlewood's rule assumes two product classes (e.g.,full- and discount-fare airline tickets, full- and discount-rate for hotel rooms, etc.) with associated prices $p_f$ and $p_d$, where $p_f > p_d$. The capacity is $C$. We ignore calcellations, no-shows, and overbooking. $D_f$ and $D_d$ denote demands for full-fare and discount-fare, respectively. The demand is a random variable with distribution $F_f$ and $F_d$.

The problem we are trying to solve is to determine how much discount class demand to accept before seeing the realization of full class demand. Why is this the case? In general hotel, airline bookings exihibit similar patterns: demands for discount class arrive first in the booking horizon and demands for the full class arrive later. This is because of two very different customer segments: leisure travelers and business travelers. Their characteristics determine such an interesting arrival pattern. Leisure travelers are price sensitive and tend to book well in advance whereas business travelers are relatively less price sensitive (afterall they are not paying for the ticket or room) and tend to book much closer to the service date. 

With those characteristics, the problem for the company is to determine how much discount class demand to accept before realizing full class demand. What happens if they don't evalaute it carefully? Let's imagine two scenarios:
- Booking limit for the discount class is too low. In this situation, more rooms reserved for late-arriving full class business customers. But if the full class demand is not strong, then the airplane will take off with more empty seats. They could have sold it to the discount class early on when the booking requests arrive. So this causes __spoilage__. Once the airplane take off, the empty seats are spoiled because airlines cannot store it and save for later days.
- Booking limie for the discount class is too high. In htis situation, less rooms are reserved for the late-arriving high paying business class. If the business demand is strong, since they sell too many rooms upfront for the discount class, they forgo the higher revenue they could have earned if they had enough seats for business class. This causes dilution because airlines earn lower price instead of higher price.

The challenge is then to balance the tradeoff with the goal of maximizing expected revenue.

Then, question is how we do it? Well, we do marginal analysis. Suppose that we have $x$ units of capacity remaining and we receive a booking request for a discount class. Two possible outcomes depending on what we decide to do:
- If we accept it, we collect __guaranteed__ revenue $p_d$.
- If we do not accept it, we are hoping that we could sell it to the higher paying class. But it is not guaranteed. We could sell this unit $x$ (marginal $x^{th}$ unit) if and only if demand for full class is at least $x$; if the demand for the full class is lower than $x$, we got nothing from it - spoilage of the seat. 

So the question is what is the expected revenue we could collect from this $x^{th}$ unit? The answer depends on the probability of full class demand being greater than or equal to $x$

$$EV(\text{reserving } x^{th} \text{ seat}) = p_f Pr(D_f \geq x)$$

Now I can make statements as follows:
- We will accept the discount class booking request if $p_d \geq p_f\Pr(D_f \geq x)$
- We will not accept the discount class booking request if $p_d < p_f\Pr(D_f \geq x)$

## Algorithm for Computing Optimal Discoount Booking Limits
The marginal analysis we just did suggests that we can use a simple iterative algorithm to calculate the optimal booking limit $b^*$ for discount class.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats
from scipy.stats import norm

In [3]:
# initial parameter values
cap = 146
full_price, disc_price = 174, 114

mu_full, sig_full = 92, 30
mu_disc, sig_disc = 80, 25

In [7]:
# For a given booking limit, calculate the revenue change
def ev_b_to_b_plus_one(bl):
    ev_b_to_b_plus_one = disc_price - full_price * (1 - norm.cdf(cap - bl, mu_full, sig_full))
    return ev_b_to_b_plus_one

In [12]:
# Algorithm
b = 0
b_opt = 0

if b == cap:
    b_opt = b
else:    
    if (ev_b_to_b_plus_one(b) <= 0) or (norm.cdf(b + 1, mu_disc, sig_disc) == 1):
        b_opt = b
    else:
        while (ev_b_to_b_plus_one(b) > 0) and (norm.cdf(b + 1, mu_disc, sig_disc) < 1):
            b += 1
        b_opt = b
        
print(b_opt)

66
